In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import sys
sys.path.append('..')

from src import models, data
from src.attributelens.attributelens import Attribute_Lens
import src.attributelens.utils as lens_utils
from src.operators import JacobianIclMeanEstimator
import numpy as np
from src.utils import experiment_utils, logging_utils
from baukit import Menu, show

import logging
logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.INFO,
    format = logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout
)

In [10]:
device = "cuda:0"
mt = models.load_model("gptj", device=device, fp16=True)
print(f"dtype: {mt.model.dtype}, device: {mt.model.device}, memory: {mt.model.get_memory_footprint()}")

2023-06-27 17:42:38 src.models INFO     loading EleutherAI/gpt-j-6B (device=cuda:0, fp16=True)
dtype: torch.float16, device: cuda:0, memory: 12219206136


In [38]:
prompt = mt.tokenizer.eos_token + " John Lennon is a"

In [39]:
lens = Attribute_Lens(mt=mt, top_k=10)
att_info = lens.apply_attribute_lens(
    prompt=prompt,
    relation_operator=None # Will use Identity if set to None. Basically Logit Lens
)
att_info['subject_range']= (1, att_info['subject_range'][-1]) # ignore the first EOS token
print('prediction:', att_info['nextwords'][-1])
p = lens_utils.visualize_attribute_lens(
    att_info, layer_skip=3, must_have_layers=[],
)

# p.write_image('bill_bradley_lens.pdf')
p.show(renderer='iframe')

2023-06-27 17:56:28 src.attributelens.attributelens INFO     prompt: "<|endoftext|> John Lennon is a"
prediction:  great
must_have_layers:  []
expected_answers:  []


In [55]:
dataset = data.load_dataset()

relation_names = [r.name for r in dataset]
relation_options = Menu(choices = relation_names, value = relation_names)
show(relation_options)

In [57]:
relation_name = relation_options.value
print("selected relation >> ", relation_name)

selected relation >>  person native language


In [58]:
datums = dataset.filter(relation_names=[relation_name])[0]

print(len(datums.samples))
experiment_utils.set_seed(12) # set seed to a constant value for sampling consistency

train, test = datums.split(5)
print("\n".join([sample.__str__() for sample in train.samples]))

919
2023-06-27 18:01:16 src.utils.experiment_utils INFO     setting all seeds to 12
Herbert Edward Read -> English
Orane Demazis -> French
Beatrix of the Netherlands -> Dutch
Aleksandr Kaleri -> Russian
Nie Weiping -> Chinese


In [59]:
mean_estimator = JacobianIclMeanEstimator(
    mt=mt,
    h_layer=8,
    beta=2.5  # magnifying the `J` contribution
) 

operator = mean_estimator(train)

In [60]:
lens = Attribute_Lens(mt=mt, top_k=10)
att_info = lens.apply_attribute_lens(
    prompt=prompt,
    relation_operator=operator,
)

print('prediction:', att_info['nextwords'])

att_info['subject_range']= (1, att_info['subject_range'][-1]) # ignore the first EOS token
p = lens_utils.visualize_attribute_lens(
    att_info, layer_skip=3, must_have_layers=[], colorscale='greens', expected_answers=['music', 'beatles']
)

p.show(renderer="iframe")

2023-06-27 18:01:43 src.attributelens.attributelens INFO     prompt: "<|endoftext|> John Lennon is a"
prediction: ['Q', ',', ',', ' a', ' great']
must_have_layers:  []
expected_answers:  ['music', 'beatles']
